In [1]:
import os, sys, matplotlib, pylab
import numpy  as np
from astropy.io import fits
from matplotlib import pyplot as plt
import re
from astropy.table import Table, Column
from astropy.io import ascii
import scipy.stats as ss

def norm(a):
    p = (a-np.median(a))/np.median(a)
    return p
plt.switch_backend('Qt5Agg')

from astropy.stats import LombScargle

#Working with EPIC sorted values, except mentioned otherwise

In [2]:
#start with path = /home/vatsal/UThesis/K2/K2_Refined/SC

def finalfilelist(path):
    os.chdir(path) # Change Directory
    nowpath = os.getcwd()
    filenamelist = filter(lambda x: x.startswith('ktwo'), os.listdir(nowpath))
    writelist = [ i +'\n' for i in filenamelist]
    with open('list.txt','w') as f:
        map( f.write , writelist)
    f.close()                                       #making the folder list

    Q10up = 'list.txt'
    Q10_file = file(Q10up).readlines()

    ktwoID = []
    ktwofilelist = []

    ktwo_detID = []
    ktwo_detfilelist = []

    for i in Q10_file:
        pathn = i.replace('\n','')
        os.chdir(path + "/%s"%pathn)

        foldername = i.strip().split(' ')
        folderlistfile = '%s_list.txt' %foldername[0]


        #1 Read names of all files ending with .fits
        nowpath = os.getcwd()
        filenamelist = filter(lambda x: x.endswith('.fits'), os.listdir(nowpath))

        #1.5 Write Raw data file into txt list (.fits)
        writelist = [ i +'\n' for i in filenamelist]
        with open('%s_Fits_list.txt' %foldername[0],'w') as f:
            map(f.write , writelist)

        #2 Read needed file data
        Kepler = '%s_Fits_list.txt' %foldername[0]
        LC_file= file(Kepler).readlines()



        for j in LC_file:
            if(j.startswith('ktwo')):
                a = re.findall('\d+', j)
                ktwoID.append(a[0])
                ktwofilelist.append(j)

            elif(j.startswith('EPIC')):
                a = re.findall('\d+', j)
                ktwo_detID.append(a[0])
                ktwo_detfilelist.append(j)

    com_ID =  [i for i in set(ktwoID) & set(ktwo_detID)]      #list of IDs common to both original and detrended list of files-
                                                              #essentially the ones that have to be analysed




    #Make a list of tuples containing names of common files from both folder

    ktwofilelist = zip([np.float(k) for k in ktwoID],ktwofilelist)
    ktwo_detfilelist = zip([np.float(l) for l in ktwo_detID],ktwo_detfilelist)


    ktwofilelist.sort(key=lambda x: x[0])
    ktwo_detfilelist.sort(key=lambda x: x[0])

    print len(ktwofilelist)
    print len(ktwo_detfilelist)

    comlist = zip([k[1] for k in ktwofilelist],[k[1] for k in ktwo_detfilelist])

    return comlist



In [6]:
#Start with nowpath = /home/vatsal/UThesis/K2/K2_Refined/SC
def plotprot(nowpath,comlist):
    numeric_const_pattern = r"""
    [-+]? # optional sign
    (?:
    (?: \d* \. \d+ ) # .1 .12 .123 etc 9.1 etc 98.1 etc
    |
    (?: \d+ \.? ) # 1. 12. 123. etc 1 12 123 etc
     )
    # followed by optional exponent part if desired
    (?: [Ee] [+-]? \d+ ) ?
    """
    rx = re.compile(numeric_const_pattern, re.VERBOSE)
    rot=[]
    for k,l in comlist:

        keplerid = re.findall('\d+',k)[0]
        k = k.replace('\n','')
        data1 = fits.open(nowpath+'/ktwo/'+k)
        dat=data1[1].data
        pdcsap_flux = dat['PDCSAP_FLUX']
        time_raw = dat['TIME']

        mnan1 = -np.isnan(pdcsap_flux)


        l = l.replace('\n','')
        data2 = fits.open(nowpath+'/ktwo-detrended/'+l)
        dat=data2[1].data
        time_det = dat['time']
        flux_det = dat['flux']
        trend_t = dat['trtime']
        trend_p = dat['trposi']
        x_cor = dat['x']
        y_cor = dat['y']

        mnan2 = -np.isnan(flux_det)

        output_file = l.replace('\n','') + '.rdb'
        fdata = Table([time_det,flux_det, trend_t, trend_p], names=['jdb', 'flux', 'trendt','trendp'])
        ascii.write(fdata,nowpath+'/ktwo-detrended/'+output_file , format='rdb')
        rot.append((keplerid,rx.findall(data2[1].header['KER_HPS1'])[2]))


        fluxerr = np.full((len(time_det[mnan2])),0.001)
        frequency, power = LombScargle(time_det[mnan2], 100.*norm(flux_det[mnan2])-100.*norm(trend_p[mnan2]),\
                                       dy=fluxerr ,center_data=False).autopower()


        fig, ax = plt.subplots(3,sharex=True)
#         #Plotting the x and y pixel centre values
#         ax[0].plot(time_det[mnan2],x_cor[mnan2],'r.')
#         ax[0].plot(time_det[mnan2],y_cor[mnan2],'k.')
#         ax[0].set_ylabel('X(red) & Y(black)')
#         ax[0].set_title(keplerid+' '+'Rotation period:'+rx.findall(data2[1].header['KER_HPS1'])[2])
#         ax[0].axhline(y=0,xmin=0,xmax=1,color='k')

        #Plotting the Raw flux from original ktwo files
    #     ax[1].plot(time_raw[mnan1],100.*norm(pdcsap_flux[mnan1]),'k.-')
    #     ax[1].set_ylabel('Raw Relative flux[%]')


        #Plotting the supposedly detrended flux from the EPIC files
        ax[0].plot(time_det[mnan2],100.*norm(flux_det[mnan2]),'k.-')
        ax[0].plot(time_det[mnan2],100.*norm(trend_t[mnan2])+100.*norm(trend_p[mnan2]),'r')
        ax[0].set_ylabel('f($\Delta F/F$)[%]')
        ax[0].set_title('EPIC'+keplerid)
        
        #Plotting the light curves with corrections for systematics
        ax[1].plot(time_det[mnan2],100.*norm(flux_det[mnan2])-100.*norm(trend_p[mnan2]),'k.-')
        ax[1].plot(time_det[mnan2],100.*norm(trend_t[mnan2]),'r')
        ax[1].set_ylabel('f-m(X)($\Delta F/F$)[%]')
#         ax[1].set_xlabel('Time[BJD-2454833]')
        
        #Plotting the detrended light curves with both systematics and time trends removed
        ax[2].plot(time_det[mnan2],100.*norm(flux_det[mnan2])-100.*norm(trend_p[mnan2])-100.*norm(trend_t[mnan2]),'k.-')
        ax[2].set_ylabel('f-m(X)-m(t)($\Delta F/F$)[%]')
        ax[2].set_xlabel('Time[BJD-2454833]')
        
        
#         ax[3].plot(1/frequency, power)
#         ax[3].set_xlabel(r'Period(days)')
#         ax[3].set_ylabel(r'Power(Normalized)')
#         ax[3].set_xscale('log')
        
#         #Plotting the light curves with corrections for both systematics and intrinsic stellar variations
#         plt.plot(time_det[mnan2],100.*norm(flux_det[mnan2])-100.*norm(trend_p[mnan2])-100.*norm(trend_t[mnan2]),'k.-')
#         plt.ylabel('f-m(X)-m(t)($\Delta F/F$)[%]')
#         plt.xlabel('Time[BJD-2454833]')
#         plt.title('EPIC'+keplerid)
        
        
        plt.show()


    #return rot


In [26]:
#First create rdblist
#Also provide the path to catfile with rotflag

##First test with one file

def flaredetector(path_to_ktwodetdir,rdblist,catfile):
    binsize = 4

    infolist = []            #to store duration of observation(in days), peak flare amp and energy, and number of flares

    energylist = []

    allflareamp = []

    durlist = []
    
    for s in rdblist:
        
    

        f = open(path_to_ktwodetdir+'/'+s, 'r')  

        d = f.read()
        table = ascii.read(d)
        time_det = np.float64(table['jdb'])
        flux_det = np.float64(table['flux'])
        trend_t = np.float64(table['trendt'])
        trend_p = np.float64(table['trendp'])

        timeint = max(time_det)-min(time_det)

        f.close()


        #mask the nan point
        # masknan = -np.isnan(data['flux'])
        # time = data['time'][masknan]
        # flux = data['flux'][masknan]/np.mean(data['flux'][masknan])
        # flux_raw = data['flux'][masknan]/np.mean(data['flux'][masknan])


        mnanm = -np.isnan(flux_det)
        time = time_det[mnanm]
        flux_d = 100.*norm(flux_det[mnanm])-100.*norm(trend_p[mnanm])-100.*norm(trend_t[mnanm])
        flux_r = 100.*norm(flux_det[mnanm])-100.*norm(trend_p[mnanm])-100.*norm(trend_t[mnanm])

        #make empty array as same length as masked flux
        fluxC = np.zeros(len(flux_d))


        #separate gap when time step > 3*time step 
        separationtime = []
        separationtime.append(0)
        for i in range(len(time)-1):
            if time[i+1] - time[i] > 3*(time[2] - time[1]):
                separationtime.append(i)
        separationtime.append(len(time))

        #print separationtime

        #do j times
        for j in range(2):
            for k in range(len(separationtime)-1):
                #take 10 point median and replace it
                for i in range(separationtime[k],separationtime[k+1]):
                    if i >= (separationtime[k]+binsize) and i <= (separationtime[k+1]-binsize): 
                        fluxC[i] = np.median(flux_d[i-binsize:i+binsize])
                    else :
                        fluxC[i] = flux_d[i]


                #Observed subtract Caculate 

                fluxdiff = flux_d-fluxC

                #move out the peak value > 1 sigma
                maskpeak = fluxdiff > 1.0*fluxdiff.std()

                #if the peak larger than standard deviation then replace the number by 10 point median
                for n, i in enumerate(maskpeak):
                    if i:
                        if n >= (separationtime[k]+binsize) and n <= (separationtime[k+1]-binsize):
                            flux_d[n] = np.median(flux_d[n-binsize:n+binsize])
                        else:
                            flux_d[n] = flux_d[n]


        #Plotting the O-C flux

        OminusC = flux_r - fluxC

        #plt.plot(time,flux_raw,'o-')
        #plt.plot(time,fluxC,'o-')
        #     plt.plot(time,OminusC,'.-')
        #     plt.show()

        flare = np.where(OminusC > 3.0*OminusC.std())

        #############

        OminusC = flux_r-fluxC

        flares = np.zeros(len(OminusC))

        flareselection = OminusC > 3.0*OminusC.std()
        #print 3.0*OminusC.std()
        #print 3.0*fluxC.std()

        for n, i in enumerate(flareselection):
            if i:
                if OminusC[n-2] > 0.05*OminusC[n]:
                    flares[n-2] = OminusC[n-2]

                if OminusC[n-1] > 0.05*OminusC[n]:
                    flares[n-1] = OminusC[n-1]

                flares[n] = OminusC[n]

                if OminusC[n] > OminusC[n+1] or OminusC[n+1] > 0.05*OminusC[n]:
                    flares[n+1] = OminusC[n+1]

                if OminusC[n+1] > OminusC[n+2] or OminusC[n+2] > 0.05*OminusC[n]:
                    flares[n+2] = OminusC[n+2]

                if OminusC[n+2] > OminusC[n+3] or OminusC[n+3] > 0.05*OminusC[n]:
                    flares[n+3] = OminusC[n+3]

                if n+4 < len(flareselection):
                    if OminusC[n+3] > OminusC[n+4] or OminusC[n+4] > 0.05*OminusC[n]:
                        flares[n+4] = OminusC[n+4]

                if n+5 < len(flareselection):
                    if OminusC[n+4] > OminusC[n+5] or OminusC[n+5] > 0.05*OminusC[n]:
                        flares[n+5] = OminusC[n+5]

        #change all minus value to 0

        for i in range(len(flares)):
            if flares[i] < 0:
                flares[i] = 0

        #remove fake flares

        for i in range(len(flares)-1):        
            if  flares[i] < 3.0*fluxC.std() and \
                flares[i-1] == 0 and \
                flares[i+1] == 0 :
                    flares[i] = 0   
            elif \
                flares[i] < 3.0*fluxC.std() and \
                flares[i-2] == 0 and \
                flares[i+1] == 0 :
                    flares[i] = 0
            elif \
                flares[i] < 3.0*fluxC.std() and \
                flares[i-1] == 0 and \
                flares[i+2] == 0 :
                    flares[i] = 0
        for i in range(len(flares)-1):        
            if  flares[i-1] == 0 and \
                flares[i+1] == 0 :
                    flares[i] = 0
        
        
        #Reading the stellar properties for flare energy calculation
        tab = Table.read(catfile,format='ascii.tab')
        z = [i for i,k in enumerate(tab['EPIC']) if str(k)==re.findall('\d+', s)[0]]
        R = tab['Radius'][z]
        Rsun = 6.96e08
        sigma = 5.67e-8         
        Teff = tab['Teff'][z]
        L = 4*np.pi*((R*Rsun)**2)*sigma*(Teff**4)*(10**7)
        

        #counting flares and find their position
        flarecount = []
        flareposition = []
        flareamp = []

        def findlocalmax(queue):
            maximum = 0
            for i in range(len(queue)):
                if (queue[i] > maximum):
                    maximum = queue[i]
                    max_position = len(queue) - i
            return maximum, max_position

        is_zero = 1
        queue = []
        dur=[]

        for i in range(len(flares)):                              #second if execs when first flare point is encountered,switches is_zero to 0
            if ((not is_zero) and flares[i] == 0):                #the whole flare is queued till the last point(when it drops to zero)
                is_zero = 1                                       #then the first if execs, does the analysis for the whole flare,
                t2 = time[i]                                                  #empties the queue, and switches is_zero to 1, goes to next point(queue.append)
                #print (findlocalmax(queue))                     #executed only in the next iteration
                #find flareposition
                index = i - findlocalmax(queue)[1]
                flareposition.append(index)
                flareamp.append(findlocalmax(queue)[0])

                #sum the each point of the data in one flare
                total = sum(queue)
                flarecount.append(np.log10(total*30*60*L/100.))      #each point is 30 minutes long
                duration = t2-t1
                dur.append(duration)
                queue = []
                continue
            if (is_zero and flares[i] != 0):
                is_zero = 0
                t1=time[i-1]
            queue.append(flares[i])

        #print flareposition
        #print flarecount
        #print L
        #print len(flareposition)

        energylist.append(flarecount)
        allflareamp.append(flareamp)
        durlist.append(dur)
        
        if(len(flarecount)!=0):
            peak_E = max(flarecount)
            peak_A = max(flares)
        else:
            peak_E = 0.
            peak_A = 0.
        
        VP = tab['VP'][z]
        P_rot = tab['P_rot'][z]
        tup = (re.findall('\d+', s)[0],timeint,len(flareposition),peak_A,peak_E,P_rot,VP)
        infolist.append(tup)
        print s

#         plt.plot(time,flares,'k.-')
#         plt.xlabel('Time[d]')
#         plt.ylabel(r'$\Delta F/F[\%]$')
#         plt.title(s)
#         plt.show()

        #     plt.plot(time,100.*norm(flux_det[mnanm])-20,'b.-')
        #     plt.plot(time,100.*norm(flux_det[mnanm])-100.*norm(trend_p[mnanm]),'k.-')

        #     plt.plot(time,flux_d+20,'g.-')


        #     plt.show()

    return infolist,energylist,allflareamp,durlist


In [4]:
#Making a list of all sc data targets and extracting the respective info from cat to make another subset cat

#Run this to get sclist and then run plotprot

sclist = finalfilelist('/home/vatsal/UThesis/K2/K2_Refined/SC')
print sclist
dat = Table.read('/home/vatsal/UThesis/K2/K2_Refined/apjs522918t5_mrt.txt',format='ascii.cds')
dat.sort('Teff')

ind = [i for i,k in enumerate(dat['EPIC']) for j in sclist if str(k)==re.findall('\d+', j[0])[0]]

print ind


30
30
[('ktwo206019387-c03_llc.fits\n', 'EPIC_206019387_mast.fits\n'), ('ktwo206050032-c03_llc.fits\n', 'EPIC_206050032_mast.fits\n'), ('ktwo206135809-c03_llc.fits\n', 'EPIC_206135809_mast.fits\n'), ('ktwo206169988-c03_llc.fits\n', 'EPIC_206169988_mast.fits\n'), ('ktwo206262336-c03_llc.fits\n', 'EPIC_206262336_mast.fits\n'), ('ktwo210317378-c04_llc.fits\n', 'EPIC_210317378_mast.fits\n'), ('ktwo210489654-c04_llc.fits\n', 'EPIC_210489654_mast.fits\n'), ('ktwo210499476-c04_llc.fits\n', 'EPIC_210499476_mast.fits\n'), ('ktwo210758829-c04_llc.fits\n', 'EPIC_210758829_mast.fits\n'), ('ktwo210894955-c04_llc.fits\n', 'EPIC_210894955_mast.fits\n'), ('ktwo210942999-c04_llc.fits\n', 'EPIC_210942999_mast.fits\n'), ('ktwo211046195-c04_llc.fits\n', 'EPIC_211046195_mast.fits\n'), ('ktwo211069418-c04_llc.fits\n', 'EPIC_211069418_mast.fits\n'), ('ktwo211081477-c04_llc.fits\n', 'EPIC_211081477_mast.fits\n'), ('ktwo211082433-c04_llc.fits\n', 'EPIC_211082433_mast.fits\n'), ('ktwo211112686-c04_llc.fits\n', 

In [5]:
#Check equality 
print sorted(dat['EPIC'][ind])
print sorted([int(re.findall('\d+', i[0])[0]) for i in sclist])

[206019387, 206050032, 206135809, 206169988, 206262336, 210317378, 210489654, 210499476, 210758829, 210894955, 210942999, 211046195, 211069418, 211081477, 211082433, 211112686, 211129272, 211400847, 211480655, 211892034, 211906940, 211945363, 212029094, 212104761, 212144340, 212178513, 212285603, 212518629, 212692562, 212820594]
[206019387, 206050032, 206135809, 206169988, 206262336, 210317378, 210489654, 210499476, 210758829, 210894955, 210942999, 211046195, 211069418, 211081477, 211082433, 211112686, 211129272, 211400847, 211480655, 211892034, 211906940, 211945363, 212029094, 212104761, 212144340, 212178513, 212285603, 212518629, 212692562, 212820594]


In [25]:
#Appending the rotation period column to the cat for addition of corresponding YN flags (Run only once)

# rot = plotprot('/home/vatsal/UThesis/K2/K2_Refined/SC',sclist)
print rot
KID, P_rot = map(list, zip(*rot))
print P_rot

t=dat[:][ind]
t.sort('EPIC')
ascii.write(t,'/home/vatsal/UThesis/K2/K2_Refined/SC/sc-md.cat', format='tab')
t.add_column(Column(P_rot,name='P_rot'),index=1)
# ascii.write(t,'/home/vatsal/UThesis/K2/K2_Refined/SC/sc-md-rotflag.cat', format='tab')


[('206019387', '4.50000000e+01'), ('206050032', '1.24261038'), ('206135809', '4.28105495e-01'), ('206169988', '-2.60820705'), ('206262336', '9.83315382e+00'), ('210317378', '28.03377087'), ('210489654', '4.81774237e-01'), ('210499476', '8.58655850e-01'), ('210758829', '4.54615478e-01'), ('210894955', '7.25480006e-01'), ('210942999', '1.84199028e+00'), ('211046195', '2.18735646e-01'), ('211069418', '8.17923030e-01'), ('211081477', '1.93405356'), ('211082433', '0.35858636'), ('211112686', '7.63966181e-01'), ('211129272', '4.88718736'), ('211400847', '5.75856158e+00'), ('211480655', '30.10721374'), ('211892034', '4.49996229e+01'), ('211906940', '0.08770702'), ('211945363', '1.76074548e+01'), ('212029094', '32.13357041'), ('212104761', '-4.36740983e+00'), ('212144340', '2.79150619e+01'), ('212178513', '-1.94595931'), ('212285603', '18.95538537'), ('212518629', '4.02792181e+01'), ('212692562', '1.20960778'), ('212820594', '1.32621415e-01')]
['4.50000000e+01', '1.24261038', '4.28105495e-01',

In [7]:
label_size = 10
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size 
plotprot('/home/vatsal/UThesis/K2/K2_Refined/SC',sclist[0:10])


/home/vatsal/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: DeprecationWarning: numpy boolean negative, the `-` operator, is deprecated, use the `~` operator or the logical_not function instead.
/home/vatsal/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:37: DeprecationWarning: numpy boolean negative, the `-` operator, is deprecated, use the `~` operator or the logical_not function instead.


In [27]:
#Analyzing flares
rdblist=['EPIC_206050032_mast.fits.rdb','EPIC_206262336_mast.fits.rdb']
path_to_ktwodetdir = '/home/vatsal/UThesis/K2/K2_Refined/SC/ktwo-detrended'
catfile = '/home/vatsal/UThesis/K2/K2_Refined/SC/sc-md-rotflag.cat'
infolist,energylist,allflareamp,durlist = flaredetector(path_to_ktwodetdir,rdblist,catfile)

/home/vatsal/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:42: DeprecationWarning: numpy boolean negative, the `-` operator, is deprecated, use the `~` operator or the logical_not function instead.


EPIC_206050032_mast.fits.rdb
EPIC_206262336_mast.fits.rdb


In [38]:
print len(durlist[0])
print len(allflareamp[0])
#(re.findall('\d+', s)[0],timeint,len(flareposition),peak_A,peak_E,P_rot,VP)

21
21
